In [1]:
import numpy as np
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
N_EPOCHS = 5
N_TRIALS = 3
BATCH_SIZE = 64

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [3]:
with open("models_hyperparameters.yaml", "r") as stream:
    models_hyperparameters = yaml.safe_load(stream)
    
x, y = utils.load_data("../data/cifar-10-batches-py/data_batch_1")
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.8, random_state=42)

train_size = x_train.shape[0]

In [4]:
assessment_model = ClassifierModel(output_shape=10, **models_hyperparameters["assessment_model"])

assessment_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)
        
assessment_model.fit(x_train, y_train, epochs=10, batch_size=BATCH_SIZE)

Epoch 1/10
32/32 [==============================] - 1s 9ms/step - loss: 2.5842 - accuracy: 0.1155
Epoch 2/10
32/32 [==============================] - 0s 10ms/step - loss: 2.3504 - accuracy: 0.1625
Epoch 3/10
32/32 [==============================] - 0s 10ms/step - loss: 2.1789 - accuracy: 0.2070
Epoch 4/10
32/32 [==============================] - 0s 10ms/step - loss: 2.0655 - accuracy: 0.2260
Epoch 5/10
32/32 [==============================] - 0s 10ms/step - loss: 2.0133 - accuracy: 0.2540
Epoch 6/10
32/32 [==============================] - 0s 10ms/step - loss: 1.9424 - accuracy: 0.2655
Epoch 7/10
32/32 [==============================] - 0s 10ms/step - loss: 1.8850 - accuracy: 0.3000
Epoch 8/10
32/32 [==============================] - 0s 10ms/step - loss: 1.8536 - accuracy: 0.3225
Epoch 9/10
32/32 [==============================] - 0s 10ms/step - loss: 1.8236 - accuracy: 0.3320
Epoch 10/10
32/32 [==============================] - 0s 11ms/step - loss: 1.8037 - accuracy: 0.3375


In [10]:
y_pred_assessment = assessment_model.predict(x_train)

losses_assessment = utils.calculate_loss_per_sample(y_train, y_pred_assessment, loss)
order_assessment = np.argsort(losses_assessment)

63/63 [==============================] - 0s 2ms/step


In [13]:
results = {}

test_models = ["test_model_1", "test_model_2", "test_model_3"]

In [16]:
for test_model in test_models:
    print(test_model)
    model_scores = []

    for _ in tqdm(range(N_TRIALS)):
        model = ClassifierModel(output_shape=10, **models_hyperparameters[test_model])
        
        model.compile(
            optimizer='adam',
            loss=tf.keras.losses.SparseCategoricalCrossentropy(),
            metrics=['accuracy']
        )
        
        for i in range(N_EPOCHS):
            n_samples = int(train_size / (N_EPOCHS - i))
            samples_ids = order_assessment[:n_samples]
            
            model.fit(x_train[samples_ids], y_train[samples_ids], epochs=1, batch_size=BATCH_SIZE, verbose=0)
        
        _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=0)
        
        model_scores.append(accuracy)
    
    results[test_model] = model_scores

test_model_1


100%|██████████| 3/3 [00:04<00:00,  1.58s/it]


test_model_2


100%|██████████| 3/3 [00:04<00:00,  1.51s/it]


test_model_3


100%|██████████| 3/3 [00:04<00:00,  1.63s/it]
